In [1]:
import random

In [2]:
class Die:
    """
    Represents a die object.
    """
    def __init__(self, value=None):
        self.value = value

    def __repr__(self):
        return str("Die: " + str(self.value))

    def roll(self):
        self.value = random.randint(1, 6)

In [3]:
class Player:
    """
    Represents a player in the game.
    """
    def __init__(self, name):
        self.name = name
        self.dice = [Die() for i in range(6)]

    def __repr__(self):
        return_string = ""
        return_string += str("Name: ") + self.name + " "
        return_string += str([d.value for d in self.dice])
        return return_string

    def get_name(self):
        return self.name
    
    def get_dice(self):
        return self.dice

    def roll_dice(self):
        for die in self.dice:
            die.roll()

    def lose_dice(self):
        self.dice.pop()

    def gain_dice(self):
        self.dice.append(Die())

    def make_bid(self, previous_bid=None):
        # Maybe a try/catch to stop erroneous bids (e.g. someone bid's a string etc) ??
        quantity = int(input("Bid quantity: "))
        value = int(input("Bid value: "))
        bid = Bid(quantity, value)

        # Beginning of a bidding round, no previous bid
        if previous_bid == None : return bid


        if bid > previous_bid : 
            return bid
        else : 
            # Continue to recursively try to make a higher bid. Can I do this?
            self.make_bid(self, previous_bid)

    def call(self, previous_bid):
        pass # To implement

    def has_lost(self):
        return not(self.dice)

    def is_remaining(self):
        return bool(self.dice)

In [4]:
class Bid:
    def __init__(self, quantity, value):
        self.quantity = quantity # {1 -> n} where n is number of dice in play
        self.value = value # {1, 2, 3, 4, 5, 6}

    def __eq__(self, other):
        return self.quantity == other.quantity and self.value == other.value

    def __gt__(self, other):
        if self.notAceBid() and other.notAceBid() :
            if self.quantity == other.quantity :
                return self.value > other.value
            else :
                return self.quantity > other.quantity
        if self.isAceBid() and other.notAceBid() : 
            return self.quantity * 2 >= other.quantity
        if self.notAceBid() and other.isAceBid() : 
            return self.quantity > other.quantity * 2
        if self.isAceBid() and other.isAceBid() :
            return self.quantity > other.quantity

    def __lt__(self, other):
        if self.notAceBid() and other.notAceBid() : 
            if self.quantity == other.quantity :
                return self.value < other.value
            else :
                return self.quantity < other.quantity
        if self.isAceBid() and other.notAceBid() : 
            return self.quantity * 2 <= other.quantity
        if self.notAceBid() and other.isAceBid() :
            return self.quantity < other.quantity * 2
        if self.isAceBid() and other.isAceBid() :
            return self.quantity < other.quantity

    def isAceBid(self):
        return self.value == 1

    def notAceBid(self):
        return self.value != 1

In [5]:
class Call:
    """
    Represents when a player calls a bid- ending that bidding round and starting the comparisons.
    """
    def __init__(self):
        pass

In [6]:
class Game():
    """
    Represents an actual instance of the game Liar's Dice (Dudo)
    """
    def __init__(self):
        self.players = []
        self.generate_players()

    def __repr__(self):
        return str(self.players)
        
    def generate_players(self):
        num_players = int(input("How many players are playing?"))
        
        # Create the Player objects and store them inside players list
        for i in range(num_players):
            name = input("Player's name?")
            self.players.append(Player(name))

    def is_finished(self):
        return self.players_remaining() < 2

    def get_players(self):
        return self.players

    def players_remaining(self):
        remaining = []
        for player in self.players:
            remaining.append(bool(player.is_remaining()))
        return sum(remaining)

In [ ]:
game = Game()

In [ ]:
game.players_remaining()

In [10]:
prev_bid = Bid(5, 5)
player = Player("Marcus")
player.make_bid(prev_bid)

my_bid > prev_bid

False

In [ ]:
game.get_players()[0].gain_dice()

In [ ]:
game.is_finished()

In [ ]:
# Main class
players = []

num_players = int(input("How many players are playing?"))
print(type(num_players))

for i in range(num_players):
    name = input("Player's name?")
    players.append(Player(name))

In [ ]:
# To do list

# Ensure logic of lt and gt for bids is correct (Write tests????)
# Set up the Game class which is going to be an instance of an actual game of Liar's dice
# Test equals case for gt and lt

# Game class

# Fields
# players field which stores the Player objects (list of Player() objects)
# game_finished - boolean True/False


# isFinished()
# notFinished()



# Collection of Player objects


# Main method??

# set up the players
# put players inside the game object
# while game.notFinished() main game loop


# Maybe a Round class?? which describes or is a single round of the game.

#  Fix the repr for the Player Class

